# 환율 뉴스 분류(직접/간접/무관)

# 0. Library

In [1]:
import os, time, json, csv, re, hashlib, threading, random, logging
import pandas as pd
from openai import OpenAI
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

True

# 1.  분류

In [1]:
# ====== API/모델 설정 ======
API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_ID = "gpt-4o-mini"

# ====== 파일 경로 ======
CSV_PATH = "naver_finance_news_2020.csv"          # 입력
YEAR_OUT = "naver_finance_news_2020_labeled.csv"  # 출력(연도 단위)
ALL_OUT  = "naver_finance_news_labeled_all_0.csv"   # 전체 누적

# ====== 기타 설정 ======
CHUNK_FLUSH   = 50
RETRY_MAX     = 5
RETRY_BACKOFF = 2.0
TIMEOUT_SEC   = 60
MAX_CHARS     = 4000    # 본문 truncate 길이

# ====== 로깅 ======
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("label_news_category.log")]
)
log = logging.getLogger("label")

In [2]:
def _truncate(text: str, max_chars: int = MAX_CHARS) -> str:
    if text is None:
        return ""
    t = str(text)
    if len(t) <= max_chars:
        return t
    head = t[: int(max_chars * 0.6)]
    tail = t[-int(max_chars * 0.35):]
    return head + "\n...\n" + tail

def _extract_json_obj(txt: str) -> str:
    """
    응답 문자열에서 첫 번째 JSON 객체만 안전하게 추출.
    코드블록/잡텍스트가 섞여도 동작.
    """
    if not isinstance(txt, str):
        return "{}"
    start, end = txt.find("{"), txt.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = txt[start:end+1]
        m = re.search(r"\{(?:[^{}]|(?R))*\}", candidate, flags=re.DOTALL)
        return m.group(0) if m else candidate
    return txt

def _row_key(row: dict) -> str:
    s = f"{row.get('date','')}\t{row.get('title','')}\t{row.get('url','')}\t{row.get('content','')}"
    return hashlib.md5(s.encode("utf-8")).hexdigest()

def load_existing_keys(csv_path: str):
    """
    기존 결과 CSV에서 해시 키 세트 로드(중복 스킵용).
    결과 파일이 없거나 깨졌으면 빈 세트 반환.
    """
    if not os.path.exists(csv_path):
        return set()
    try:
        prev = pd.read_csv(csv_path, encoding="utf-8-sig", engine="python")
        keys = set()
        for _, r in prev.iterrows():
            base_row = {
                "date": r.get("date", ""),
                "title": r.get("title", ""),
                "url": r.get("url", ""),
                "content": r.get("content", "")
            }
            keys.add(_row_key(base_row))
        return keys
    except Exception:
        return set()

In [3]:
_client = None
def get_client():
    """OpenAI 클라이언트 lazy init (커널 리셋/셀 순서 꼬임 방지)"""
    global _client
    if _client is None:
        if not API_KEY:
            raise RuntimeError("API_KEY가 비어있습니다. Cell 1에서 API_KEY를 설정하세요.")
        _client = OpenAI(api_key=API_KEY)
    return _client

class RateLimiter:
    """최소 요청 간격 강제 → QPS 제한"""
    def __init__(self, min_interval_sec=1.0):
        self.min_interval = float(min_interval_sec)
        self._lock = threading.Lock()
        self._next_ok = 0.0
    def wait(self):
        with self._lock:
            now = time.time()
            if now < self._next_ok:
                time.sleep(self._next_ok - now)
            self._next_ok = time.time() + self.min_interval

rl = RateLimiter(min_interval_sec=1.0)   # 필요시 1.5~2.0으로 올리세요

def _get_retry_after_seconds(e) -> float:
    """429 응답의 Retry-After(초) 또는 reset 헤더를 읽어 대기시간 계산."""
    try:
        h = getattr(e, "response", None).headers or {}
    except Exception:
        h = {}
    v = h.get("retry-after") or h.get("Retry-After") or h.get("x-ratelimit-reset-requests") or h.get("X-RateLimit-Reset-Requests")
    try:
        return float(v) if v is not None else 5.0
    except Exception:
        return 5.0

def _jitter(base: float, frac: float = 0.3) -> float:
    """풀 지터: base ± (base*frac)"""
    delta = base * frac
    return max(0.0, base + random.uniform(-delta, delta))

In [4]:
SYSTEM_PROMPT = """너는 금융 뉴스 분류기다. 입력으로 뉴스의 date, title, url, content가 주어진다.

### category(분류) 정의
- FX-Direct: 원/달러, 환율, 외환시장, 달러 강세/약세 등 환율을 직접 언급하는 기사
- FX-Indirect: 환율 단어가 없어도 환율에 구조적으로 영향을 주는 기사 (금리, 물가, 무역·경상수지, 통화정책, 글로벌 리스크, 원자재 가격, 지정학적 사건 등)
- Irrelevant: 환율과 직접·간접 관련이 없는 기사
- 타국/북한 환율 기사는 한국 원/달러와의 구조적 연결이 서술될 때만 FX-Indirect, 그렇지 않으면 Irrelevant.

### 출력 형식(반드시 JSON 한 객체):
{"category": "FX-Direct" | "FX-Indirect" | "Irrelevant", "confidence": 0.xxx}
JSON 외 텍스트는 절대 출력하지 말 것.
"""

USER_TEMPLATE = """뉴스 기사:
date: {date}
title: {title}
url: {url}
content: {content}

분류 결과 JSON만 출력하라.
"""

def classify_category(row: dict) -> dict:
    date = str(row.get("date", ""))
    title = str(row.get("title", ""))
    url = str(row.get("url", ""))
    content = _truncate(row.get("content", ""))

    user_prompt = USER_TEMPLATE.format(date=date, title=title, url=url, content=content)
    messages = [{"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}]

    delay = RETRY_BACKOFF
    for attempt in range(1, RETRY_MAX + 1):
        try:
            # QPS 제한
            rl.wait()

            client = get_client()
            resp = client.chat.completions.create(
                model=MODEL_ID,
                messages=messages,
                temperature=0.0,
                max_tokens=64,
                response_format={"type": "json_object"},  # JSON 강제
                timeout=TIMEOUT_SEC,
            )
            txt = resp.choices[0].message.content.strip()
            # 직파싱 → 실패 시 JSON만 추출하여 재파싱
            try:
                data = json.loads(txt)
            except json.JSONDecodeError:
                data = json.loads(_extract_json_obj(txt))

            category = str(data.get("category", "")).strip()
            confidence = float(data.get("confidence", 0.0))
            if category not in ("FX-Direct", "FX-Indirect", "Irrelevant"):
                category = "Irrelevant"

            return {
                "date": date,
                "title": title,
                "url": url,
                "content": content,
                "category": category,
                "confidence": confidence,
            }

        except Exception as e:
            # 429 등 레이트리밋 우선 처리
            msg = str(e).lower()
            is_rl = ("429" in msg) or ("too many requests" in msg) or ("rate" in msg and "limit" in msg)
            if is_rl:
                server_wait = _get_retry_after_seconds(e)
                sleep_sec = _jitter(max(server_wait, delay))
                print(f"[429] rate limited → wait {sleep_sec:.2f}s (retry-after≈{server_wait:.2f}, attempt {attempt}/{RETRY_MAX})", flush=True)
                time.sleep(sleep_sec)
                delay = min(delay * 2.0, 60.0)
                continue

            print(f"[API 재시도] attempt={attempt}/{RETRY_MAX}, error={e}, backoff={delay}s", flush=True)
            if attempt == RETRY_MAX:
                return {
                    "date": date,
                    "title": title,
                    "url": url,
                    "content": content,
                    "category": "Irrelevant",
                    "confidence": 0.0,
                    "error": str(e),
                }
            time.sleep(delay)
            delay = min(delay * 2.0, 60.0)

In [5]:
def main():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"입력 파일이 없습니다: {CSV_PATH}")
    # 읽기 안전 옵션
    df = pd.read_csv(CSV_PATH, encoding="utf-8-sig", engine="python")
    df = df.rename(columns={c: c.strip().lower() for c in df.columns})
    required = {"date", "title", "url", "content"}
    assert required.issubset(df.columns), f"CSV에 {required} 컬럼이 필요합니다. 누락: {required - set(df.columns)}"

    log.info(f"[시작] 입력 파일: {CSV_PATH}, 행수={len(df)}")

    year_existing_keys = load_existing_keys(YEAR_OUT)
    all_existing_keys  = load_existing_keys(ALL_OUT)
    log.info(f"[상태] 중복키 수: year={len(year_existing_keys)}, all={len(all_existing_keys)}")

    # tqdm 총량 계산
    def _needs(row):
        key = _row_key({"date":row["date"], "title":row["title"], "url":row["url"], "content":row["content"]})
        return key not in year_existing_keys and key not in all_existing_keys
    total_todo = int(df.apply(_needs, axis=1).sum())
    log.info(f"[계획] 신규 라벨 대상: {total_todo}건")

    buffer, processed, skipped = [], 0, 0
    year_header_needed = not os.path.exists(YEAR_OUT)
    all_out_exists     = os.path.exists(ALL_OUT)

    last_beat = time.time()
    BEAT_SEC  = 15  # 하트비트 주기(초)

    with tqdm(total=total_todo, desc="라벨링 진행", unit="doc") as pbar:
        for idx, r in df.iterrows():
            row_in = {
                "date": r.get("date", ""),
                "title": r.get("title", ""),
                "url": r.get("url", ""),
                "content": r.get("content", ""),
            }
            key = _row_key(row_in)
            if key in year_existing_keys or key in all_existing_keys:
                skipped += 1
            else:
                if processed % 10 == 0:
                    log.info(f"[호출] idx={idx}, title={row_in['title'][:40]}...")

                data = classify_category(row_in)
                buffer.append(data)
                processed += 1

                pbar.update(1)
                pbar.set_postfix({
                    "done": processed,
                    "skip": skipped,
                    "cat": data.get("category", "NA"),
                    "conf": f"{data.get('confidence', 0.0):.2f}"
                })

                # 부분 저장
                if len(buffer) >= CHUNK_FLUSH:
                    part = pd.DataFrame(buffer)
                    part.to_csv(
                        YEAR_OUT, mode="a", index=False, encoding="utf-8-sig",
                        header=year_header_needed, quoting=csv.QUOTE_ALL, lineterminator="\n"
                    )
                    year_header_needed = False
                    part.to_csv(
                        ALL_OUT, mode="a", index=False, encoding="utf-8-sig",
                        header=(not all_out_exists), quoting=csv.QUOTE_ALL, lineterminator="\n"
                    )
                    all_out_exists = True
                    buffer = []
                    log.info(f"[중간 저장] 누적 처리 {processed}건, 스킵 {skipped}건")

            # 하트비트
            now = time.time()
            if now - last_beat >= BEAT_SEC:
                log.info(f"[HB] 진행중… 처리 {processed} / {total_todo}, 스킵 {skipped}")
                last_beat = now

    # 잔여 저장
    if buffer:
        part = pd.DataFrame(buffer)
        part.to_csv(
            YEAR_OUT, mode="a", index=False, encoding="utf-8-sig",
            header=year_header_needed, quoting=csv.QUOTE_ALL, lineterminator="\n"
        )
        part.to_csv(
            ALL_OUT, mode="a", index=False, encoding="utf-8-sig",
            header=(not all_out_exists), quoting=csv.QUOTE_ALL, lineterminator="\n"
        )
        log.info(f"[최종 저장] 추가 {len(part)}건 저장")

    log.info(f"[완료] 처리 {processed}건, 스킵 {skipped}건")
    log.info(f"- 연도 파일: {YEAR_OUT}")
    log.info(f"- 전체 누적: {ALL_OUT}")

main()

2025-09-26 01:07:03,576 [INFO] [시작] 입력 파일: naver_finance_news_2020.csv, 행수=7174
2025-09-26 01:07:03,771 [INFO] [상태] 중복키 수: year=2545, all=2545
2025-09-26 01:07:03,858 [INFO] [계획] 신규 라벨 대상: 4632건


라벨링 진행:   0%|          | 0/4632 [00:00<?, ?doc/s]

2025-09-26 01:07:03,917 [INFO] [호출] idx=1535, title=경제학자들 "석달후 살아남을 기업 없어…50조원 갖곤 태부족"...
2025-09-26 01:07:06,475 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:08,588 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:09,976 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:12,957 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:13,942 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:15,128 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:17,064 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:17,947 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:07:19,00

[429] rate limited → wait 8.61s (retry-after≈9.00, attempt 1/5)


2025-09-26 01:09:52,881 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:09:52,906 [INFO] [HB] 진행중… 처리 33 / 4632, 스킵 2542
2025-09-26 01:09:53,112 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 01:09:53,113 [INFO] Retrying request to /chat/completions in 8.640000 seconds
2025-09-26 01:10:02,004 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 01:10:02,005 [INFO] Retrying request to /chat/completions in 8.640000 seconds
2025-09-26 01:10:11,564 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:10:11,571 [INFO] [HB] 진행중… 처리 34 / 4632, 스킵 2542
2025-09-26 01:10:12,407 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 01:10:13,784 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2025-09-26 09:54:49,253 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:49,264 [INFO] [HB] 진행중… 처리 4556 / 4632, 스킵 2542
2025-09-26 09:54:50,317 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:51,209 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:52,168 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:53,249 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:53,259 [INFO] [호출] idx=7102, title=역대 최초 2,800선 돌파한 코스피...
2025-09-26 09:54:54,319 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:54:54,619 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 09:54:54,623 [INFO] Retrying request to /chat/completion

[429] rate limited → wait 11.24s (retry-after≈9.00, attempt 1/5)


2025-09-26 09:57:52,751 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:57:52,757 [INFO] [HB] 진행중… 처리 4577 / 4632, 스킵 2542
2025-09-26 09:57:52,968 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 09:57:52,969 [INFO] Retrying request to /chat/completions in 8.640000 seconds
2025-09-26 09:58:01,841 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 09:58:01,844 [INFO] Retrying request to /chat/completions in 8.640000 seconds
2025-09-26 09:58:11,746 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:58:11,754 [INFO] [HB] 진행중… 처리 4578 / 4632, 스킵 2542
2025-09-26 09:58:12,801 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 09:58:13,720 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 

[429] rate limited → wait 10.70s (retry-after≈9.00, attempt 1/5)


2025-09-26 10:00:46,682 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 10:00:46,687 [INFO] [HB] 진행중… 처리 4597 / 4632, 스킵 2542
2025-09-26 10:00:47,655 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 10:00:48,778 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 10:00:49,579 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 10:00:49,589 [INFO] [중간 저장] 누적 처리 4600건, 스킵 2542건
2025-09-26 10:00:49,589 [INFO] [호출] idx=7142, title="북한 당국, 국내 시장서 외화 사용 금지"<아사히>...
2025-09-26 10:00:50,701 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 10:00:51,007 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-26 10:00:51,008 [INFO] Retrying request to /chat/completions in 8.640000 seconds
2025-09-26 10:01:00